<a href="https://colab.research.google.com/github/AliAbyaneh/Extracting-Image-from-EEG-signals/blob/master/CNN_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

#@title Default title text
import os
import zipfile
cwd = os.getcwd()
print(cwd)
os.chdir("../../")
os.chdir("../gdrive/My Drive")
cwd = os.getcwd()
print(cwd)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/content
/gdrive/My Drive


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

!pip install scipy
from scipy import signal

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np


EPOC_Brain_location = {"AF3" : 0,
                  "F7"  : 1,
                  "F3"  : 2,
                  "FC5" : 3,
                  "T7"  : 4,
                  "P7"  : 5,
                  "O1"  : 6,
                  "O2"  : 7,
                  "P8"  : 8, 
                  "T8"  : 9,
                  "FC6" : 10,
                  "F4"  : 11,
                  "F8"  : 12,
                  "AF4" : 13}
MW_Brain_location = {"FP1" : 0}
N_Samples_device = {"MindWave" : 1024,
          "EPOC"     : 256 
          }
def Load_data(infile, device, data_size = 100, event_start_point = 67635, split_data = False):
    if device == "EPOC":
        Brain_location = EPOC_Brain_location
    elif device == "MindWave":
        Brain_location = MW_Brain_location
    N_locations = len(Brain_location)
    test_size = int(0.2 * data_size)
    train_size = data_size - test_size
    N_data = N_Samples_device[device]
    if N_locations > 1:
        arr = np.zeros([data_size, N_locations, N_data])
    else:
        arr = np.zeros([data_size, N_data], dtype = 'int32')
    label = np.zeros([data_size], dtype = 'int32')
    for i in range(N_locations*data_size):
        
        temp = infile.readline()
        if len(temp) < 10:
            break
        x = temp.split()
        header = x[0:6]
        event = int(header[1]) - event_start_point
        channel = Brain_location[header[3]]
        temp = x[6].split(',')
        while len(temp) < N_data:
            temp.append('0')
        n = int(header[4])
#        print(n)
        if N_locations > 1:
            arr[i//N_locations][channel] = list(map(float,temp))[:N_data]
        else:
            arr[i//N_locations] = list(map(float,temp))[:N_data]
        label[i//N_locations] = n
    if split_data == True:
        return arr[0:train_size], label[0:train_size], arr[train_size:data_size], label[train_size:data_size]
    return arr, label
def EPOC_Load_data(infile, data_size = 100, event_start_point = 67635):
    N_locations = 14
    test_size = int(0.2 * data_size)
    train_size = data_size - test_size
    N_data = 256
    arr = np.zeros([data_size, N_locations, N_data])
    label = np.zeros([data_size], dtype = 'int32')
    for i in range(N_locations*data_size):
        
        temp = infile.readline()
        if len(temp) < 10:
            break
        x = temp.split()
        header = x[0:6]
        event = int(header[1]) - event_start_point
        channel = EPOC_Brain_location[header[3]]
        temp = x[6].split(',')
        while len(temp) < N_data:
            temp.append('0')
        n = int(header[4])
        if(n != -1):
            arr[event][channel] = list(map(float,temp))[:N_data]
            label[event] = n
    return arr,label

def get_specific_data(arr, labels, l1, l2):
    data = [[] for i in range(10)]
    for i in range(len(arr)):
        data[labels[i]].append(arr[i].transpose())
    length = 0
    for i in range(10):
        length = length + len(data[i])
    length = length // 4
    tags = []
    content = []
    for i in range(length):
        if i < len(data[l1]):
            tags.append(0)
            content.append(data[l1][i])
        elif i < len(data[l1]) + len(data[l2]):
            tags.append(1)
            content.append(data[l2][i - len(data[l1])])
        else:
            tags.append(2)
    for i in range(10):
        if i != l1 and i != l2:
            for j in range(len(data[i]) // 7):
                content.append(data[i][j])
    while len(tags) < len(content):
      tags.append(2)
    return content, tags
    
infile = open("EP1.01.txt")
data, labels = EPOC_Load_data(infile, data_size = 40000)
content, tags = get_specific_data(data[:38000], labels[:38000], 2, 5)
del data
del labels
data = np.asarray(content)
labels = np.asarray(tags)

In [4]:
print(data.shape)

(11993, 256, 14)


In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.20, shuffle= True)

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=np.shape(x_train)[1:]))
model.add(tf.keras.layers.Conv1D(128, 11, padding='same', activation='tanh'))
model.add(tf.keras.layers.AveragePooling1D(pool_size=(4)))
model.add(tf.keras.layers.Dropout(0.5))



model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(256, 11, padding='same', activation='tanh'))
model.add(tf.keras.layers.AveragePooling1D(pool_size=(2)))
model.add(tf.keras.layers.Dropout(0.25))


# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv1D(128, 8, padding='same', activation='elu'))
# model.add(tf.keras.layers.AveragePooling1D(pool_size=(4)))
# model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dense(3))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc (None, 256, 14)           56        
_________________________________________________________________
conv1d (Conv1D)              (None, 256, 128)          19840     
_________________________________________________________________
average_pooling1d (AveragePo (None, 64, 128)           0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 128)           0         
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 64, 128)           512       
_________________________________________________________________
conv1d_1 (Conv1D)    

In [7]:
print(np.shape(x_train))
xx = x_train[:,0,:]
print(xx.shape)

(9594, 256, 14)
(9594, 14)


In [0]:
import os
import datetime

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
tpu_model.compile(
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=2e-3, ),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['sparse_categorical_accuracy', 'accuracy']
)

# model.save('CNN1D_model_EPOC.h5')  # creates a HDF5 file 'my_model.h5'
filepath = "./CNN1D_model_EPOC_s25/weights25-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkPointer = tf.keras.callbacks.ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=False, mode='auto', monitor='val_acc')
logdir="./CNN1D_model_EPOC_s25/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorBoard = tf.keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True, write_grads=False, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')
def train_gen(batch_size):
  while True:
    offset = np.random.randint(0, train_data.shape[0] - batch_size)
    yield train_data[offset:offset+batch_size], train_labels[offset:offset + batch_size]
    

History = tpu_model.fit(
    x = x_train,
    y = y_train,
    batch_size = 256,
    epochs=3000,
    validation_data = (x_valid, y_valid),
    callbacks = [checkPointer,tensorBoard]
)
import pickle
with open('./CNN1D_model_EPOC/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(History.history, file_pi)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.39.41.146:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 5818258760656821809)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9718759055896099161)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7434626406905760444)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1,